In [1]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
df = pd.read_csv('../../main_dataset/data_all.csv')

weather_df = pd.read_csv('../../main_dataset/context_weather_all_processed.csv', parse_dates=['data'])
weather_df.rename({'data' :'date'}, inplace=True, axis=1)
weather_df.set_index('date', inplace=True)

pop_df = pd.read_csv('../../main_dataset/context_poi_popularity.csv', index_col='Unnamed: 0')
pop_df.rename(columns={'temp_discr' :'temp'}, inplace=True)

fest_df = pd.read_csv("../../main_dataset/context_festivities.csv", index_col=0, parse_dates=True)
fest_df.reset_index(inplace=True)
fest_df.rename({"index" : "date",
                "values" : "festivity"}, axis=1, inplace=True)

/tmp/ipykernel_51228/2116522632.py:1: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../main_dataset/data_all.csv')


In [3]:
df

,id_veronacard,profilo,data_visita,ora_visita,sito_nome,poi
0,04AA72F2185080,vrcard-24-2019,2019-08-22,17:03:00,Torre dei Lamberti,59
1,0419C5E2185081,vrcard-48-2019,2019-12-01,11:46:00,Torre dei Lamberti,59
2,042C0FE2185085,vrcard-48-2019,2019-07-31,13:37:00,Torre dei Lamberti,59
3,042B2DE2185085,vrcard-48-2019,2019-07-31,13:37:00,Torre dei Lamberti,59
4,04191AE2185081,vrcard-48-2019,2019-12-01,11:46:00,Torre dei Lamberti,59
...,...,...,...,...,...,...
4448589,0440ADE2185081,vrcard-48-2019,2020-01-01,10:56:07,Giardino Giusti,75
4448590,04161EE2185081,vrcard-48-2019,2020-01-01,10:55:40,Giardino Giusti,75
4448591,044452E2185081,vrcard-48-2019,2020-01-01,10:55:27,Giardino Giusti,75
4448592,043412E2185085,vrcard-48-2019,2020-01-01,10:22:41,Giardino Giusti,75


In [ ]:
df['date'] = df['data_visita'] + ' ' + df['ora_visita']
df = df[['sito_nome', 'poi', 'date']]
df['date'] = pd.to_datetime(df.loc[:, 'date'])

In [ ]:
df.info()

In [ ]:
df['sito_nome'].unique()

In [ ]:
df_dic = {}
df_count_dic = {}
for poi in df['sito_nome'].unique():
    mask = df['sito_nome'] == poi
    df_dic[poi] = df.loc[mask]
    #df_dic[poi].reset_index(drop=True, inplace=True)
    df_dic[poi].set_index('date', inplace=True)
    df_count_dic[poi] = df_dic[poi].resample('15min').size().reset_index(name='count')
    
    new_index = pd.date_range(
    start=weather_df.index.min(),
    end=weather_df.index.max() + pd.Timedelta(days=1) - pd.Timedelta(minutes=15),
    freq='15min', name='date'
    )
    df_meteo = pd.DataFrame(index=new_index)
    df_meteo = df_meteo.merge(weather_df, left_index=True, right_index=True, how='left')
    df_meteo.ffill(inplace=True)
    df_count_dic[poi] = df_count_dic[poi].merge(df_meteo, on='date', how='left')
    
    #df_popularity = pop_df[pop_df['poi_name'] == poi]
    #df_count_dic[poi] = df_count_dic[poi].merge(df_popularity, on=['rain', 'temp'], how='left').drop(['poi_name', 'poi'], axis=1)

    df_count_dic[poi] = df_count_dic[poi].merge(fest_df, on='date', how="left")

    df_count_dic[poi]['rolling_mean_30d'] = df_count_dic[poi]['count'].shift(96).rolling(window=2880, min_periods=2880).mean()
    df_count_dic[poi]['above_avg'] = (df_count_dic[poi]['count'] > df_count_dic[poi]['rolling_mean_30d']).astype(int)
    df_count_dic[poi]['rolling_mean_30d'].fillna(0, inplace=True)
    
    df_count_dic[poi] = df_count_dic[poi].replace({'no_rain': 0, 'rain': 1})
    #mask_2020 = (df_count_dic[poi].date < pd.to_datetime('2020-01-01')) & (df_count_dic[poi].date >= pd.to_datetime('2019-01-01'))
    #mask_2020 = (df_count_dic[poi].date < pd.to_datetime('2020-01-01')) 
    #df_count_dic[poi] = df_count_dic[poi][mask_2020]

In [ ]:
df_count_dic['Casa di Giulietta'].head()

In [ ]:
df_count_dic['Casa di Giulietta'].tail()

In [ ]:
for poi in df_count_dic.keys():
    if len(df_count_dic[poi]) == 0:
        pass
    else:
        df_string = poi.lower().replace(' ', '_')
        #df_count_dic[poi].to_csv(f'../../main_dataset/count_data_train/data_{df_string}_train.csv')
        #df_count_dic[poi].to_csv(f'../../main_dataset/count_data_test/data_{df_string}_test.csv')
        #df_count_dic[poi].to_csv(f'../../main_dataset/count_data_2020/data_{df_string}_2020.csv')
        df_count_dic[poi].to_csv(f'../../main_dataset/count_data/data_{df_string}.csv')